<a href="https://colab.research.google.com/github/arzow-m/iubdc/blob/main/data_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

5 june 2025
- look for patients with ADRs and record subject_id
- gather all relevant data for those ids

relevant data:
- subject id (only for patients with adrs, so will be the same)
- age -> higher age increases^ risk of adr
- diagnosis(or # of diagnoses?) ->drug-disease interaction^ (diabetes, high/low bp, ulcer, glaucoma, enlarged prostate, poor bladder control, and insomnia)
    -# of diagnoses (count unique icd per subject_id)
- medications
    -# of types -> use of serveral medications ^ (more likely to have 1+ chronic medical disorder, liver/kidneys do not work as well, etc)
        https://www.merckmanuals.com/en-ca/home/older-people-s-health-issues/aging-and-medications/aging-and-medications#Benefits-and-Risks-of-Prescription-Medications_v838778
    -prescription?
- number of adr?


last
- filter ages to be within range
- rename columns

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# load mimic-iv tables
df_patient = pd.read_csv('/patients.csv.gz', index_col=0, compression='gzip')
df_diagnose = pd.read_csv('/diagnoses_icd.csv.gz', index_col=0, compression='gzip')
df_prescription = pd.read_csv('/prescriptions.csv.gz', index_col=0, compression='gzip')
df_procedure = pd.read_csv('/procedures_icd.csv.gz', index_col=0, compression='gzip')
df_chartevents = pd.read_csv('/chartevents.csv.gz', index_col=0, compression='gzip')

# filter for elderly patients
df_patient = df_patient[df_patient['anchor_age'] >= 65]
df_mimic = df_patient[['gender', 'anchor_age']].copy()
df_mimic.reset_index(inplace=True)  # now has subject_id column

# diagnoses
diagnosis_counts = df_diagnose[df_diagnose.index.isin(df_mimic['subject_id'])] \
    .groupby('subject_id')['icd_code'].nunique().reset_index(name='num_unique_diagnoses')
df_mimic = df_mimic.merge(diagnosis_counts, on='subject_id', how='left')

# prescriptions
drug_counts = df_prescription[df_prescription.index.isin(df_mimic['subject_id'])] \
    .groupby('subject_id')['drug'].nunique().reset_index(name='num_unique_drugs')
drug_counts['polypharmacy'] = (drug_counts['num_unique_drugs'] >= 5).astype(int)
df_mimic = df_mimic.merge(drug_counts, on='subject_id', how='left')

# procedures
procedure_counts = df_procedure[df_procedure.index.isin(df_mimic['subject_id'])] \
    .groupby('subject_id')['icd_code'].nunique().reset_index(name='num_unique_procedures')
df_mimic = df_mimic.merge(procedure_counts, on='subject_id', how='left')

# lab averages
# define common lab item IDs (replace with actual MIMIC item IDs if different)
lab_ids = {
    'creatinine': 50912,
    'potassium': 50971
}

# adr label (from chart events - itemid 227968)
adr_event = df_chartevents[(df_chartevents['itemid'] == 227968) & (df_chartevents['value'] == "No")]
adr_patients = adr_event.index.unique().tolist()
df_mimic['had_adr'] = df_mimic['subject_id'].isin(adr_patients).astype(int)

# handle missing values
df_mimic.fillna({
    'num_unique_diagnoses': 0,
    'num_unique_drugs': 0,
    'num_unique_procedures': 0,
}, inplace=True)

# gender
le = LabelEncoder()
df_mimic['gender'] = le.fit_transform(df_mimic['gender'])  # F = 0, M = 1

# export final csv
df_mimic.to_csv('clean_mimic_extended.csv', index=False)
print(df_mimic.head())


   subject_id  gender  anchor_age  num_unique_diagnoses  num_unique_drugs  \
0    10003400       0          72                   113               123   
1    10002428       0          80                    84                97   
2    10037928       0          78                    94                85   
3    10020640       0          91                    21                35   
4    10019003       0          65                    82               117   

   polypharmacy  num_unique_procedures  had_adr  
0             1                   24.0        1  
1             1                   14.0        0  
2             1                    4.0        0  
3             1                    0.0        1  
4             1                   15.0        0  


# condensing all patients into one unique row for model training

In [ ]:
from sklearn.preprocessing import LabelEncoder

# convert 'had_ADR' into binary
df_mimic['had_adr'] = df_mimic['had_adr'].astype(object) # cast the column so float error goes away
adr_flags = df_mimic['subject_id'].isin(adr_patients)
df_mimic.loc[adr_flags, 'had_adr'] = 'Yes'
df_mimic.loc[~adr_flags, 'had_adr'] = 'No'

df_mimic['had_adr'] = df_mimic['had_adr'].astype(str).str.strip().str.capitalize()
df_mimic['had_adr'] = df_mimic['had_adr'].map({'Yes': 1, 'No': 0})

# print(df_mimic['had_adr'].value_counts(dropna=False))
# print(df_mimic['had_adr'].unique())

df_grouped = df_mimic.groupby('subject_id').agg({
    'gender': 'first', # doesnt change
    'anchor_age': 'first', #doesnt change
    'diagnosis icd': pd.Series.nunique, # num of unique diagnoses
    'drug': pd.Series.nunique, # num of unique drugs
    'procedure icd': pd.Series.nunique, # num of unique procedures
    'had_adr': 'max' # doesnt change
}).reset_index()

# rename the columns for consistency
df_grouped.rename(columns = {
    'diagnosis icd': 'num_unique_diagnoses',
    'drug': 'num_unique_drugs',
    'procedure icd': 'num_unique_procedures'
}, inplace = True)

# turn gender into numerical value for consistency
le = LabelEncoder()
df_grouped['gender'] = le.fit_transform(df_grouped['gender'])
# print(le.classes_)shows that F = 0, M = 1

print(df_grouped.head())
df_mimic.to_csv('df_grouped.csv', index=False)

KeyError: "Column(s) ['diagnosis icd', 'drug', 'procedure icd'] do not exist"